### Librería Folium para visualizar mapas

- Documentación https://python-visualization.github.io/folium/index.html

In [19]:
import folium
import pandas as pd
from folium.plugins import HeatMap
import requests
from folium.plugins import MeasureControl

In [20]:
# Dataset con localizaciones de Starbucks en el mundo

data_starbucks = pd.read_csv('./starbucks_worldwide.csv')

In [21]:
# Filtramos los starbucks que están en España

starbucks_spain = data_starbucks[data_starbucks['Country']=='ES']

In [22]:
starbucks_spain.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
5100,Starbucks,24637-238504,Paseo de la Castellana,Licensed,"Paseo de la Castellana 278, Bonaire Shopping C...",Madrid,AB,ES,28046,NaN,GMT+1:00 Europe/Madrid,-3.69,40.48
5101,Starbucks,15307-148964,Bonaire,Licensed,CC Bonaire Nacional II Km 345,Aldaia,AN,ES,46960,963073401,GMT+1:00 Europe/Madrid,-0.48,39.46
5102,Starbucks,6115-147505,Malaga Airport Landside,Licensed,Avenida Garcia Morato s/n,Malaga,AN,ES,29004,NaN,GMT+1:00 Europe/Madrid,-4.49,36.68
5103,Starbucks,24564-232744,ECI Malaga,Licensed,"Calle Armengual De La Mota, 7",Malaga,AN,ES,29007,NaN,GMT+1:00 Europe/Madrid,-4.43,36.72
5104,Starbucks,26270-244207,ECI Capricho,Licensed,Alfonso de Hohenlohe 2,Malaga,AN,ES,29602,NaN,GMT+1:00 Europe/Madrid,-4.92,36.50


In [8]:
# Nos conectamos a una api para ver cuáles son los restaurantes cercanos
BASE_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch"
key = ""

In [9]:
# La localización de partida que establezco yo en la del centro del Madrid a un radio de 1km y medio

res = requests.get("{}/json?location=40.42,-3.7&radius=1500&type=restaurant&key={}".format(BASE_URL, key)).json()

In [13]:
#Sacamos latitud y longitud del primer restaurante que nos devuelve la API

latitude_restaurant = res['results'][0]['geometry']['location']['lat']
longitude_restaurant = res['results'][0]['geometry']['location']['lng']

- #### folium.Map()

Primero creamos el mapa con el punto de partida que queramos 

- #### folium.CircleMarker() 

Pintamos todos los starbucks en el mapa y 
- #### folium.Marker() 

Marcamos el restaurante que nos devolvió la API
- #### folium.Circle() 

Hacemos un círculo con un radio de 100 metros que engloba al starbucks y el restaurante

- #### .add_to(mapa) 

Todo lo que se crea hay que añadirlo al mapa inicial que hemos creado

- #### MeasureControl()

Podemos añadir además un medidor de distancias. Está en el mapa(arriba a la derecha), se señalan dos puntos en el mapa y te dice la distancia entre ellos.

In [17]:

map_starbucks = folium.Map(location=[40.42, -3.7], width=750, height=500, zoom_start=15)

for index, row in starbucks_spain.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']],
                        radius=9,
                        popup="City: {}, latitude {}, longitude {}".format(row['City'], row['Latitude'], row['Longitude']),
                        fill_color="#F35C50",
                       ).add_to(map_starbucks)
    
    folium.Marker([latitude_restaurant, longitude_restaurant],
                        radius=2,
                        icon=folium.Icon(icon='cloud'), # Icono nube, hay más en la documentación
                        fill_color="#F35C50",
                       ).add_to(map_starbucks)
    folium.Circle(location=[40.42, -3.7],
                    radius=100
                   ).add_to(map_starbucks)
map_starbucks.add_child(MeasureControl()) 

map_starbucks.save('./map_starbucks.html') # Podemos guardar nuestro mapa como un html
map_starbucks

- ### Heatmap

Hacemos una lista de tuplas con la latitud y la longitud para pasárselo a Folium

In [23]:
values_cooordinates = list(zip(starbucks_spain.Latitude.values, starbucks_spain.Longitude.values))

In [28]:
values_cooordinates[0:5]

[(40.48, -3.69), (39.46, -0.48), (36.68, -4.49), (36.72, -4.43), (36.5, -4.92)]

In [20]:
heatmap = folium.Map([40.42, -3.7], tiles='Stamen Toner', zoom_start=6)

HeatMap(values_cooordinates).add_to(heatmap)
heatmap